주의 : my SQL에서 OD데이터 파일 생성 이후 사용!

In [1]:
import pandas as pd
import os

In [2]:
# 정류소 정보 불러오기
info = pd.read_excel('/Users/taejin/Desktop/데분 학회/급행버스/서울시버스노선별정류소정보.xlsx')

# 불필요한 컬럼 및 중복 제거
# info.drop(['NODE_ID'], axis=1, inplace=True)
info.drop_duplicates(inplace=True)

### 1. 정류장명 & 좌표 매핑

O/D data temp 파일 불러옴

In [3]:
#파일 경로 지정
directory_path = '/Users/taejin/Desktop/데분 학회/급행버스/'

# 파일 리스트에서 CSV 파일만 필터링
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# 데이터프레임 딕셔너리 생성
dataframes = {}

# 각 CSV 파일을 데이터프레임으로 읽어와서 딕셔너리에 저장
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df_name = os.path.splitext(csv_file)[0]  # 파일명에서 확장자 제거하여 데이터프레임 이름으로 사용
    df = pd.read_csv(file_path)
    df['노선명'] = df['노선명'].astype(str)
    df.rename(columns={'avg(승객수)' : '승객수'}, inplace=True)
    dataframes[df_name] = df


정류장명 매핑

In [4]:
# 노선명 ARS_ID 딕셔너리
info_dict = info[['ARS_ID', '정류소명']].set_index('ARS_ID')['정류소명'].to_dict()

# 정류장명 매핑
for df_name in list(dataframes.keys()):
    df = dataframes[df_name]
    df['승차정류장'] = df['승차_정류장ARS'].map(info_dict)
    df['하차정류장'] = df['하차_정류장ARS'].map(info_dict)

좌표 매핑

In [5]:
# ARS_ID, 좌표 딕셔너리
info_gpsX = info[['ARS_ID', 'X좌표']].set_index('ARS_ID')['X좌표'].to_dict()
info_gpsY = info[['ARS_ID', 'Y좌표']].set_index('ARS_ID')['Y좌표'].to_dict()

# 좌표 매핑
for df_name in list(dataframes.keys()):
    df = dataframes[df_name]
    df['승차정류장_X좌표'] = df['승차_정류장ARS'].map(info_gpsX)
    df['승차정류장_Y좌표'] = df['승차_정류장ARS'].map(info_gpsY)
    df['하차정류장_X좌표'] = df['하차_정류장ARS'].map(info_gpsX)
    df['하차정류장_Y좌표'] = df['하차_정류장ARS'].map(info_gpsY)
    

노선명 route id 매핑

In [6]:
# route_id 노선명 딕셔너리
routeID_dict = info[['노선명', 'ROUTE_ID']].set_index('노선명')['ROUTE_ID'].to_dict()

# route id 매핑
for df_name in list(dataframes.keys()):
    df = dataframes[df_name]
    df['ROUTE_ID'] = df['노선명'].map(routeID_dict)

### 2. 데이터 내보내기

od_temp2 데이터 내보냄

In [78]:
# 각 데이터프레임 출력 (필요 시)
for df_name, df in dataframes.items():
    print(f"DataFrame Name: {df_name}")
    # print(df.head())  # 첫 5개 행 출력

DataFrame Name: od_weekend_temp
DataFrame Name: od_weekday_temp


In [79]:
for i , data in enumerate(list(dataframes.values())):
    # csv파일로 저장
    dataframes[list(dataframes.keys())[i]].to_csv(f'/Users/taejin/Desktop/데분 학회/급행버스/{list(dataframes.keys())[i]}2.csv')

In [ ]:
# for i , data in enumerate(list(dataframes.values())):

#     #불필요한 컬럼 제거
#     # data.drop(['Unnamed: 0'], axis=1, inplace=True)

#     # 승차 정류장 좌표
#     merged_df = pd.merge(data, info, left_on=['노선명','승차_정류장ARS', '승차_정류장명'], right_on=['노선명', 'ARS_ID', '정류소명'], how='left')
#     merged_df.rename(columns={'X좌표' : '승차정류장_X좌표', 'Y좌표' : '승차정류장_Y좌표'}, inplace=True)

#     #불필요한 컬럼 제거
#     merged_df.drop(['정류소명', 'ARS_ID'], axis=1, inplace=True)

#     #하차 정류장 좌표
#     merged_df = pd.merge(merged_df, info, left_on=['노선명','하차_정류장ARS', '하차_정류장명'], right_on=['노선명', 'ARS_ID', '정류소명'], how='left')
#     merged_df.rename(columns={'X좌표' : '하차정류장_X좌표', 'Y좌표' : '하차정류장_Y좌표'}, inplace=True)

#     #불필요한 컬럼 제거
#     merged_df.drop(['정류소명', 'ARS_ID'], axis=1, inplace=True)

#     # 데이터프레임 딕셔너리에 저장
#     dataframes[list(dataframes.keys())[i]] = merged_df

#     # csv파일로 저장
#     dataframes[list(dataframes.keys())[i]].to_csv(f'/Users/taejin/Desktop/데분 학회/급행버스/{list(dataframes.keys())[i]}.csv')

### 3.승하차 정류장 순번 입력

In [ ]:
for df_name in list(dataframes.keys()):
    df = dataframes[df_name]
    
    # 승하차 정류장 순번 추가
    df['승차_정류장순번'] = ''
    df['하차_정류장순번'] = ''
    
    # 승차 하차 순번 저장
    for route_name in station_by_route_dict.keys():
        api_df = station_by_route_dict[route_name]
        # api_df = api_df[['seq', 'arsId']].fillna(-1).astype(int)
        sequence_dict = api_df[['arsId', 'seq']].fillna(-1).astype(int).set_index('arsId')['seq'].to_dict()
        
        # '승차_정류장순번' 열 추가
        df.loc[df['노선명'] == route_name, '승차_정류장순번'] = df.loc[df['노선명'] == route_name, '승차_정류장ARS'].map(sequence_dict)
        # '하차_정류장순번' 열 추가
        df.loc[df['노선명'] == route_name, '하차_정류장순번'] = df.loc[df['노선명'] == route_name, '하차_정류장ARS'].map(sequence_dict)
        # 방향 열 추가
    dataframes[df_name] = df    

### 4. 방향 및 속도 정보 추가

Sql 구문에서 api_station_by_route와 od_week_temp2 데이터 병합

### 5. 상위 20개 노선 추출

[143, 151, 1164, 2211, 3315, 5513, 5515, 5617, 5621, 6628, 7016, 7612, 7734]

In [131]:
#파일 경로 지정
directory_path = '/Users/taejin/Desktop/데분 학회/급행버스/'

# 파일 리스트에서 CSV 파일만 필터링
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# 데이터프레임 딕셔너리 생성
dataframes = {}

# 각 CSV 파일을 데이터프레임으로 읽어와서 딕셔너리에 저장
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df_name = os.path.splitext(csv_file)[0]  # 파일명에서 확장자 제거하여 데이터프레임 이름으로 사용
    df = pd.read_csv(file_path)
    df['노선명'] = df['노선명'].astype(str)
    df.rename(columns={'avg(승객수)' : '승객수'}, inplace=True)
    dataframes[df_name] = df

In [143]:
dataframes['od_weekday'][dataframes['od_weekday']['방향'].isna()]['노선명'].unique()

array(['703', '8551', '8112', '1140'], dtype=object)

In [132]:
top_20_weekday = dataframes['od_weekday']
top_20_weekend = dataframes['od_weekend']

In [ ]:
route_list = ['143', '151', '1164', '2211', '3315', '5513', '5515', '5617', '5621', '6628', '7016', '7612', '7734']

top_20_weekday = top_20_weekday[top_20_weekday['노선명'].isin(route_list)]
top_20_weekend = top_20_weekend[top_20_weekend['노선명'].isin(route_list)]

top_20_weekday.to_csv('/Users/taejin/Desktop/데분 학회/급행버스/top_20_od_weekday.csv')
top_20_weekend.to_csv('/Users/taejin/Desktop/데분 학회/급행버스/top_20_od_weekend.csv')

### 5. 시각화